In [126]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [127]:
hotel_distance_to_location = pd.read_csv("./Cleaned table/hotel_distance_to_location.csv")
hotel_facility = pd.read_csv("./Cleaned table/hotel_facility.csv")
hotel_location = pd.read_csv("./Cleaned table/hotel_location.csv")
hotel_price = pd.read_csv("./Cleaned table/hotel_price.csv")
hotel_map = pd.read_csv("./Cleaned table/hotel_map.csv")
hotel_price_daily = pd.read_csv("./Cleaned table/hotel_price_daily.csv")
hotel_service = pd.read_csv("./Cleaned table/hotel_service.csv")
hotel_quality = pd.read_csv("./Cleaned table/hotel_quality.csv")
hotel_review = pd.read_csv("./Cleaned table/hotel_review.csv")
province = pd.read_csv("./Cleaned table/province.csv")
district = pd.read_csv("./Cleaned table/district.csv")
street = pd.read_csv("./Cleaned table/street.csv")
domain = pd.read_csv("./TripI/domain.csv")
domain_city_info = pd.read_csv("./TripI/domain_city_info.csv")
hotel_logging = pd.read_csv("./Cleaned table/hotel_logging.csv")
hotel_image = pd.read_csv("./TripI/hotel_image.csv")

C:\Users\ADMIN\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\ADMIN\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (21,27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [128]:
hotel_map = pd.read_csv("./Cleaned table/hotel_map.csv")
#Xoa du lieu bi mat ca hai
drop1 = hotel_map[(hotel_map['overall_score_OTA'] == -1) & (hotel_map['overall_score'].isna())]
hotel_map = hotel_map.drop(drop1.index[:])
drop2 = hotel_map[(hotel_map['overall_score_OTA'] == -2) & (hotel_map['overall_score'].isna())]
hotel_map = hotel_map.drop(drop2.index[:])
#Doi scale 10 thanh 100
hotel_map.overall_score_OTA = hotel_map.overall_score_OTA * 10
#Dien het gia tri bị mất cua OTA
mask1 = hotel_map['overall_score_OTA'] == -10
hotel_map.loc[mask1,'overall_score_OTA'] = hotel_map.overall_score
mask2 = hotel_map['overall_score_OTA'] == -20
hotel_map.loc[mask2,'overall_score_OTA'] = hotel_map.overall_score
#Dien het gia tri bị mất cua TripI
hotel_map['overall_score'] = hotel_map['overall_score'].fillna(hotel_map['overall_score_OTA'])

In [129]:
rank = hotel_map[['domain_hotel_id', 'hotel_id', 'domain_id', 'overall_score_OTA', 'overall_score']]
rank['score_mean'] = (rank['overall_score_OTA'] + rank['overall_score'])/2

<ipython-input-129-b66c7d6dde70>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rank['score_mean'] = (rank['overall_score_OTA'] + rank['overall_score'])/2


In [130]:
temp = hotel_logging[['domain_hotel_id','action_name']]
num = temp.groupby(['domain_hotel_id', 'action_name']).size().reset_index(name='counts_search')
num1 = num[num['action_name'] == '1_search']
rank = rank.join(num1.set_index('domain_hotel_id'), on = 'domain_hotel_id')
num = num.rename(columns = {'action_name':'action_name_1','counts_search':'counts_click'})
num2 = num[num['action_name_1'] == '2_click']
rank = rank.join(num2.set_index('domain_hotel_id'), on = 'domain_hotel_id')
num = num.rename(columns = {'action_name_1':'action_name_2','counts_click':'counts_book'})
num3 = num[num['action_name_2'] == '3_book']
rank = rank.join(num3.set_index('domain_hotel_id'), on = 'domain_hotel_id')
rank = rank.drop(['action_name','action_name_1','action_name_2'], axis = 1)
rank['counts_search'] = rank['counts_search'].fillna(0)
rank['counts_click'] = rank['counts_click'].fillna(0)
rank['counts_book'] = rank['counts_book'].fillna(0)

In [131]:
a = hotel_map[['domain_hotel_id','hotel_id']]
temp = hotel_review[['domain_hotel_id', 'score', 'domain_id']]
temp = temp.merge(a, on = 'domain_hotel_id')
mask1 = temp['domain_id'] == 5
temp.loc[mask1,'score'] = temp.score * 2.5
mask2 = temp['domain_id'] == 4
temp.loc[mask2,'score'] = temp.score * 2
mask3 = temp['domain_id'] == 2
temp.loc[mask3,'score'] = (temp.score - 4.5)/5.5 * 10
temp.score = temp.score -5
temp = temp.rename(columns ={'score':'review_score'})
temp = temp.groupby(['hotel_id']).agg(np.sum)
temp = temp.reset_index()
temp = temp.drop(['domain_id','domain_hotel_id'], axis =1)
temp.T

,0,1,2,3,4,5,6,7,8,9,...,5479,5480,5481,5482,5483,5484,5485,5486,5487,5488
hotel_id,4.0,15.000000,25.0,32.0,34.0,67.0,70.0,75.0,84.0,126.000000,...,47655.0,47657.0,47658.000000,47659.0,47662.00,47663.000000,47667.0,47668.000000,47675.00,47679.0
review_score,220.0,1049.916666,692.2,3580.9,469.0,626.5,314.6,157.2,176.0,28.090909,...,5.0,20.0,153.833333,392.0,26.25,21.683333,25.4,360.683333,691.15,-1.6


In [132]:
rank = rank.join(temp.set_index('hotel_id'), on = 'hotel_id')
rank['review_score'] = rank['review_score'].fillna(0)

In [133]:
rank['SCORE'] = rank['score_mean'] * 1000 + (rank['counts_search']*1 + rank['counts_click']*20 + rank['counts_book']*400)*10 + rank['review_score']

In [134]:
rank = rank.groupby('hotel_id').agg(np.mean)
rank = rank.reset_index()
rank = rank.drop(['domain_hotel_id', 'domain_id'], axis = 1)
rank = rank.sort_values(by=['SCORE'],  ascending=False).reset_index(drop=True)

In [135]:
temp = hotel_map[['hotel_id','name','address','province_id','district_id','street_id','latitude','longitude','description']]
temp = temp.drop_duplicates()
rank = rank.merge(temp, on ='hotel_id')
rank = rank[['hotel_id','name','address','overall_score_OTA','overall_score','score_mean','counts_search','counts_click','counts_book','review_score','SCORE','province_id','district_id','street_id','latitude','longitude','description']]

In [136]:
temp = hotel_service.drop(['domain_hotel_id','hotel_id_encoded','domain_id'], axis = 1)
temp = temp.groupby('hotel_id').agg(np.max)
temp = temp.reset_index()
rank = rank.merge(temp, on ='hotel_id')

In [137]:
temp = hotel_facility.drop(['domain_hotel_id','hotel_id_encoded','domain_id'], axis = 1)
temp = temp.groupby('hotel_id').agg(np.max)
temp = temp.reset_index()
rank = rank.merge(temp, on ='hotel_id')

In [138]:
temp = hotel_price[['hotel_id','mean']]
temp = temp.groupby('hotel_id').agg(np.mean)
temp = temp.reset_index()
rank = rank.merge(temp, on = 'hotel_id')
rank = rank.rename(columns = {'mean':'price_mean'})

In [139]:
rank = rank.drop(['street_id'], axis = 1)

In [140]:
rank.T

,0,1,2,3,4,5,6,7,8,9,...,5214,5215,5216,5217,5218,5219,5220,5221,5222,5223
hotel_id,40415,40412,38408,38420,45545,1680,38437,30436,2101,40454,...,47188,37088,38770,6185,34969,46752,46068,45959,37034,47608
name,Premier Residences Phu Quoc Emerald Bay Manage...,Seashells Hotel & Spa Phú Quốc (Seashells Phu ...,Aaron Hotel,Smile Hotel Nha Trang,Green Tree Hotel Phú Quốc,Khách Sạn Senriver (Senriver Hotel),Ibis Styles Nha Trang,Cozrum Homes Trương Định,Sun River Hotel,InterContinental Phú Quốc Long Beach Resort,...,OYO 592 An Nhien Hotel & Cafe,Nancy Sweet Apartment - A2205,Hoang Ky,Phuong Ngan Hotel Ninh Binh,Nam Phuong Home - Lily room in the heart of Hanoi,Homestay Des Amis - Bạn Hữu,Our Houses Hạ Long,Touch Da Nang Hostel,Nancy Thuy Tien Apartment 1009,Anh Đào Homestay
address,"Khem Beach An Thới, Phú Quốc, Kiên Giang","Vo Thi Sau, Dương Đông, Đảo Phú Quốc, Việt Nam","Số 6 Trần Quang Khải, Thành Phố Nha Trang, Khá...","45/1 Hùng Vương, Thành Phố Nha Trang, Khánh Hòa","Khu tổ hợp du lịch Sonasea Villas & Resort, Bã...","238 Bạch Đằng, Quận Hải Châu, Đà Nẵng","86 Hùng Vương, Thành Phố Nha Trang, Khánh Hòa","73 Trương Định, Quận 3, Hồ Chí Minh","Số 132-134-136 Bạch Đằng, Quận Hải Châu, Đà Nẵng","Bãi Trường Dương Tơ, Phú Quốc, Kiên Giang",...,"71 Hà Huy Tập, P.3, Lâm Đồng, Đà Lạt","OSC Land Residence, 110 Vo Thi Sau, Thắng Tam,...","3 Pham Van Dong, Vĩnh Hòa, Nha Trang, Việt Nam","Số 6, Ngõ 48 phố 5, đường Chiến Thắng, phường ...","Quận Hoàn Kiếm, Hà Nội, Việt Nam","18A Đinh Tiên Hoàng, Tân An, Hội An","2A/26A, Anh Đào, Bãi Cháy, Quảng Ninh, Hạ Long","K134/26 Lê Hữu Trác, An Hải Đông, Q.Sơn Trà, Đ...","Thuy Tien Building, 84 Tran Phu, Phường 5, Vũn...","Lử Chô 2, Lâu Thí Ngài, Bắc Hà, Lào Cai, Bắc Hà"
overall_score_OTA,87.7083,90,86,90,94.5,80,88,84,78.0972,88.2276,...,20,20,20,20,20,20,20,20,20,10
overall_score,88,91,86,86,94,83,85,88,80,90,...,20,20,20,20,20,20,20,20,20,10
score_mean,87.8542,90.5,86,88,94.25,81.5,86.5,86,79.0486,89.1138,...,20,20,20,20,20,20,20,20,20,10
counts_search,818,1691,4591,3583,3750,3514,2585,1713,4770,2635,...,2,0,0,0,0,0,0,0,0,0
counts_click,468,187,174,131,213,56,97,198,193,76,...,0,0,0,0,0,0,0,0,0,0
counts_book,20,22,12,15,8,14,12,9,2,10,...,0,0,0,0,0,0,0,0,0,0
review_score,362,8101.9,216.818,766,419.083,2367.6,348,448.833,695,1108.75,...,-3,5,0,-3,-3,-3,-3,-3,-3,0
